In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings, HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS


In [76]:
import os
from tqdm.autonotebook import tqdm

os.environ["OPENAI_API_KEY"] = "My API Key"

In [3]:
import json
def split_to_doc(file_path, spliter):
    all_document = []
    # Load episode title
    with open("./audio_info.json", "r") as f:
        titles = json.load(f)

    for file in tqdm(os.listdir(file_path)):
        with open(file_path + file, "r") as f:
            data = json.load(f)
            for ids, transcript in data.items():
                etadata = {'title': ""}
                if ids in titles:

                    metadata = {'title': titles[ids]}
                
                text = transcript["text"]
                splitted_text = spliter.split_text(text)
                for chunk in splitted_text:

                    document = Document(page_content=chunk, metadata=metadata)
                    all_document.append(document)
    return all_document

In [66]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
)

In [67]:
doc_all = split_to_doc("./gdiy_json_data/", text_splitter)

  0%|          | 0/340 [00:00<?, ?it/s]

In [29]:
embeddings = HuggingFaceEmbeddings(model_name="dangvantuan/sentence-camembert-base")
# model =  SentenceTransformer("dangvantuan/sentence-camembert-large",device="cuda")

In [10]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator(embedding=embeddings).from_documents(doc_all[:100])

In [68]:
db = FAISS.from_documents(doc_all[:200], embeddings)

In [51]:
#db2 = FAISS.from_documents(doc_all[100:200], embeddings)
#db.merge_from(db2)

In [69]:
def batch_list(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

In [70]:
batched_doc = batch_list(doc_all, batch_size=200)
batch_len = sum(1 for _ in batch_list(doc_all, batch_size=200))

In [71]:
for i, doc in tqdm(enumerate(batched_doc), total=batch_len):
        if i == 0:
                continue
        #current_db = FAISS.from_documents(doc, embeddings)
        db.add_documents(doc)

  0%|          | 0/255 [00:00<?, ?it/s]

In [72]:
db.save_local("faiss_index")

In [73]:
retriever = db.as_retriever()

In [77]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

In [136]:
query = "Comment prendre de meilleures décisions et être plus créatif ?"
res=db.similarity_search_with_score(query=query, k=5)

In [137]:
for resp in res:

    print(resp)

(Document(page_content="vouloir améliorer le modèle, comment est-ce que c'est plus vertueux pour les élèves, comment est-ce qu'on accueille plus d'élèves avec le même niveau de qualité. Voilà, mais je me sens pas bloqué sur certains sujets. Ok. Et si tu pouvais te donner un conseil à toi il y a dix ans ? Peut-être plus souvent sortir la tête du guidon, plus souvent essayer de voir à long terme, essayer d'être meilleur en planification. Moi j'ai un côté très opérationnel, j'adore mettre les mains dans le cambouis, donc du coup c'est pas trop mon genre de passer une journée sur un business plan par exemple, pas du tout. Et au stade où est LiveVenture aujourd'hui, ça peut être intéressant justement, pas forcément l'exercice business plan mais de tracer la ligne et de se dire où est-ce que je veux être dans un an, dans deux ans, dans trois ans, c'est pas dans mon caractère. D'accord, t'as encore une vie perso ? Ouais complètement. Ouais ? Complètement. C'est bien, c'est pas donné à tous le

In [128]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end.
Give many informations as possible.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in French:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [129]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff",
                                  retriever=db.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [144]:
query = "quels conseils à une personne qui veut se lancer aujourd'hui?"

In [145]:
qa.run(query)

" La meilleure chose à faire pour une personne qui veut se lancer aujourd'hui est d'utiliser LegalStart pour choisir la bonne plateforme administrative et juridique dès le départ. Cela vous permet de vous enlever de la charge mentale inutile. De plus, LegalStart met à votre disposition 100 juristes basés en France, ainsi que 20 partenaires ultra-premium qui vous accompagnent pour la comptabilité, l'assurance et la banque."

In [95]:
from langchain import LLMChain

with open("./1.txt") as f:
    state_of_the_union = f.read()
texts = text_splitter.split_text(state_of_the_union)

In [115]:
# on peut selectionner texts[2:-3]
texts[-3]

"que deux trois bons plans. Pour finir n'oubliez pas que si vous avez des besoins en accompagnement digital, toute l'équipe de Causa Vostra va vous bichonner comme il se doit si vous nous écrivez sur contact.causavostra.com. Je suis Mathieu Stéph podcast, je vous arrête tout de suite un tout petit instant, abonnez-vous à New Work City, c'est New Work comme le travail, Work avec un W à la place du Y de New York City, et c'est le podcast de ma chère amie Julia Molkou qui est produit par Orso Media. Et si vous cherchez un podcast qui aborde en long, en large et en travers tous les sujets tabous de l'entreprise, c'est croustillant je vous préviens, ne cherchez plus, vous êtes au bon endroit. Dans New Work City, Julia et ses invités exportent les sujets tabous, les sujets qui fâchent et qu'on n'ose même pas évoquer à la machine à café. Sur le papier, la vie de bureau, vous le savez tous, c'est un long fleuve tranquille. Le cadre est bien délimité, les frontières sont définies au millimètre"

In [101]:
docs = [Document(page_content=t) for t in texts[3:10]]

In [102]:
from langchain.chains.summarize import load_summarize_chain
llm = OpenAI(temperature=0)

In [103]:
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

"\n\nSimon Davelin is an experienced entrepreneur in the digital space, having managed teams of 3, 100, and 20 people in his businesses. He started his first business at 17 with the help of older associates and his parents' support. His forum for gamers was the beginning of the trend of socializing around video games, and he has since invested in around 20 startups. He encourages others to persevere, but not for too long."